# Word Error Rate Eval
uttrs: 002, 010, 050, 150, 275, 390 

### WER calculation example
reference  = "I am 32 years old and I am a software developer" \
hypothesis = "I am a 32 year old and I am as a developer" \
Substitutions = 1, Insertions = 2, Deletions = 1 \
Number of word  = 12

WER = (1 + 2 + 1) / 12 \
WER = 0.33


In [9]:
import whisper
import pickle
import os
import librosa
import jiwer
from tqdm import tqdm

In [30]:
uttr = '150'
directory = f'/home/ytang363/7100_voiceConversion/audio_test/HifiGAN/res/{uttr}_res32_result'
text_path = '/home/ytang363/7100_voiceConversion/VCTK-Corpus-0.92/txt'

# config for jiwer
transforms = jiwer.Compose(
    [
        jiwer.ExpandCommonEnglishContractions(),
        jiwer.RemoveEmptyStrings(),
        jiwer.ToLowerCase(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
        jiwer.RemovePunctuation(),
        jiwer.ReduceToListOfListOfWords(),
    ]
)

total = 0
files = sorted(os.listdir(directory))
for filename in tqdm(files, desc="Processing audio files"):
    if os.path.isfile(os.path.join(directory, filename)):
        source = filename.split('x')[0]
        target = filename.split('x')[1].split('.')[0]

        word = os.path.join(text_path, source, f'{source}_{uttr}.txt')
        with open(word, 'r') as file:
            ref = file.read().strip()

        audio_path = os.path.join(directory, filename)
        audio, sampling_rate = librosa.load(audio_path)    
        model = whisper.load_model("base.en")
        result = model.transcribe(audio, language="en")

        hypothesis = result["text"]

        wer = jiwer.wer(
                        ref,
                        hypothesis,
                        truth_transform=transforms,
                        hypothesis_transform=transforms,
                    )        
        total += 1 - wer

Processing audio files: 100%|██████████| 36/36 [00:40<00:00,  1.11s/it]


In [31]:
total / len(files)

-0.13210978835978837

# Result
res-002: 0.19444
res-010: 0.27083
res-050: 0.2039
res-150: -0.1321
res-275:
res-390: